In [16]:
# imports an excel sheet of ConceptualEntity <> ConceptualEntity
# expects the Conceptual Entities to be fully qualified.
import sqlite3
import pandas as pd
import sys
sys.path.append('../')
from user_packages import hashing

conn = sqlite3.connect('app_db.sqlite')
cur = conn.cursor()

TargetExcelFilename = 'ConceptualEntityConceptualEntity_TypeOf.xlsx'
TargetExcelFilepath = 'imports/concept/'

In [17]:
df = pd.read_excel(TargetExcelFilepath + TargetExcelFilename)
df.fillna('', inplace=True)

df['RecordSource'] = df.apply(lambda row: 'Py.XL_CE_to_CETypeOf', axis=1)
df['ConceptualEntityConceptualEntity_TypeOfKeyPhrase'] = df.apply(lambda row: row['ConceptualEntityKeyPhrase'] + ':' + row['ConceptualEntity_TypeOfKeyPhrase'], axis=1)

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ConceptualEntityConceptualEntity_TypeOfHashKey'
  , columns = ['ConceptualEntityConceptualEntity_TypeOfKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ConceptualEntityHashKey'
  , columns = ['ConceptualEntityKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ConceptualEntity_TypeOfHashKey'
  , columns = ['ConceptualEntity_TypeOfKeyPhrase']
)


#df

In [18]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_XL_CE_to_CETypeOf")
conn.commit()
df.to_sql('stg_Py_XL_CE_to_CETypeOf', conn, if_exists='append', index=False)

27

In [19]:
# Write to the Link (ConceptualEntity - ConceptualEntity_TypeOf)
sql_query = """
INSERT INTO rv_l_ConceptualEntityConceptualEntity_TypeOf
(
  ConceptualEntityConceptualEntity_TypeOfHashKey
  , LoadDate
  , RecordSource
  , ConceptualEntityHashKey
  , ConceptualEntity_TypeOfHashKey
)
SELECT
  stg.ConceptualEntityConceptualEntity_TypeOfHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.ConceptualEntityHashKey
  , stg.ConceptualEntity_TypeOfHashKey
FROM
  stg_Py_XL_CE_to_CETypeOf stg
WHERE
  stg.ConceptualEntityConceptualEntity_TypeOfHashKey IS NOT NULL
  AND ConceptualEntityConceptualEntity_TypeOfHashKey NOT IN (
    SELECT ConceptualEntityConceptualEntity_TypeOfHashKey
    FROM rv_l_ConceptualEntityConceptualEntity_TypeOf
  )
""";
conn.execute(sql_query)
conn.commit()